In [ ]:
from importlib import reload ,import_module
import module.utilize as utilize
import module.multiVariant as multiVariant
import module.singleVariant as singleVariant
import module.multiHistogramBase as multiHistogramBase
import numpy as np
from numba import njit,jit, float32
import module.singleVariantCopulaBase as CopulaBase
from tqdm import tqdm
import time
from multiprocessing import Pool
from sklearn.metrics import root_mean_squared_error
import cupy as cp
import module.multiHistogramSparse as multiHistogramSparse
import module.ananlysisFuncion as ananlysisFunction
reload(utilize)
reload(multiVariant)
reload(singleVariant)
reload(multiHistogramBase)
reload(CopulaBase)
reload(multiHistogramSparse)

startTime=time.time()
#attribute_names=np.array(["phi_grav","particle_mass_density","zmom","ymom"])
attribute_names=np.array(["phi_grav","particle_mass_density"])
incremental_number=300
all_ensamble_data=np.empty([0,incremental_number,64,64,64])

for name in attribute_names:
    data=utilize.readFiles(name,incremental_number)
    data=data.reshape(1,incremental_number,64,64,64)
    all_ensamble_data=np.append(all_ensamble_data,data,axis=0)

#print(all_ensamble_data.shape)
#print(all_ensamble_data[0].shape)
covBlockSize=5
dataBlockSize=5
binsNumber=128
sizeZ=60
sizeY=60
sizeX=60
minMaxBlockSize=2
isMinMax=False

print("start fit model")
with tqdm(total=4, desc="Model fitting") as pbar:
    #oursModel=multiVariant.multiDistCopula3D(all_ensamble_data,dataBlockSize,covBlockSize,binsNumber,[sizeZ,sizeY,sizeX],minMaxBlockSize,isMinMax)
    oursModel=multiVariant.multiDistCopula3D.load(f"Nyx_{attribute_names.shape[0]}varaibles_{incremental_number}members_128Bins_dBlock5_cBlock2_new")
    #conditions=np.array([[0,1e5],[3e10,5e10]])
    #oursModel.fit()
    print("ours complete fit")
    pbar.update(1)
    copulaBlockSize=2
    copulaBaseModel=CopulaBase.multiVariantCopulaBase(all_ensamble_data,copulaBlockSize)
    copulaBaseModel.fit()
    print("copula complete fit")
    pbar.update(1)
    multiHistBlockSize=3
    multiHistModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=multiHistBlockSize,binsNumber=binsNumber)
    multiHistModel.fit()
    print("multi-hist complete fit")
    pbar.update(1)
    gtModel=multiHistogramSparse.multiHistogramSpaseModel(all_ensamble_data,blockSize=1,binsNumber=binsNumber)
    gtModel.fit()

    multiBinEdges=gtModel.vBinEdges

    print("complete fit")
    pbar.update(1)

oursError=[]
copulaError=[]
mtError=[]


#multiBinEdges=cp.asarray(multiBinEdges,dtype=cp.float32)


with tqdm(total=sizeZ*sizeY*sizeX, desc="總進度") as pbar:
    for idx in range(sizeZ * sizeY * sizeX):
        
        z = idx // (sizeY * sizeX)
        y = (idx // sizeX) % sizeY
        x = idx % sizeX        
        ### GroundTruth ###

        gtMultiHistModel=gtModel.getHistByPos(z,y,x)
        coords_gt, vals_gt = gtMultiHistModel.to_coo()
        ### ours method ###

        oursSamples=oursModel.sampleByPos(z,y,x)
        oursHistModel=multiHistogramSparse.SparseMultiHistogramBlock(bin_edges=multiBinEdges)
        oursHistModel.add_samples(oursSamples)
        oursHistModel.normalize()
        
        coords_target,vals_target=oursHistModel.to_coo()
        emd=ananlysisFunction.emd_sparse(coords_gt,vals_gt,coords_target,vals_target)
        oursError.append(emd)

       
        ### copula Base ###
    
        copulaSamples=copulaBaseModel.sampleByPos(z,y,x)
        copulaHistModel=multiHistogramSparse.SparseMultiHistogramBlock(bin_edges=multiBinEdges)
        copulaHistModel.add_samples(copulaSamples)
        copulaHistModel.normalize()

        coords_target,vals_target=copulaHistModel.to_coo()
        emd=ananlysisFunction.emd_sparse(coords_gt,vals_gt,coords_target,vals_target)
        copulaError.append(emd)
        

        ### multiHist ###

        mtMultiHistModel=multiHistModel.getHistByPos(z,y,x)

        coords_target,vals_target=mtMultiHistModel.to_coo()
        emd=ananlysisFunction.emd_sparse(coords_gt,vals_gt,coords_target,vals_target)
        mtError.append(emd)
        
        pbar.update(1)

start fit model


Model fitting:  25%|██▌       | 1/4 [00:00<00:00,  7.99it/s]

ours complete fit


Model fitting:  50%|█████     | 2/4 [00:50<00:58, 29.44s/it]

copula complete fit


Model fitting:  75%|███████▌  | 3/4 [01:32<00:35, 35.52s/it]

multi-hist complete fit


Model fitting: 100%|██████████| 4/4 [02:22<00:00, 35.64s/it]


complete fit


總進度: 100%|██████████| 8000/8000 [01:22<00:00, 96.59it/s] 


In [ ]:
oursError=np.array(oursError)
copulaError=np.array(copulaError)
mtError=np.array(mtError)

oursError=oursError.mean()
copulaError=copulaError.mean()
mtError=mtError.mean()

from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
Outputfilename = f"NyxEMD_{attribute_names.shape[0]}varaibles_{timestamp}.txt"
end_Time=time.time()

with open(Outputfilename , "w", encoding="utf-8") as f:
    print(f"ours error:{oursError.mean()}",file=f)
    print(f"copula error:{copulaError.mean()}",file=f)
    print(f"mt error: {mtError.mean()}",file=f)
    print(f"執行時間:{end_Time-startTime}",file=f)